In [0]:
# PySpark sample: Comprehensive workflow demonstration (100+ lines)

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
from datetime import datetime
import random


In [0]:

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Databricks Sample Workflow") \
    .getOrCreate()

# Sample data schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("department", StringType(), True),
    StructField("salary", DoubleType(), True),
    StructField("joining_date", StringType(), True)
])



In [0]:
# Generate sample data
names = ["Alice", "Bob", "Charlie", "David", "Eva", "Frank", "Grace", "Hannah", "Ian", "Jane"]
departments = ["HR", "Engineering", "Finance", "Marketing"]
data = [(i, name1s[i % len(names)], random.choice(departments), round(random.uniform(40000, 120000), 2),
         datetime(2020 + (i % 5), random.randint(1, 12), random.randint(1, 28)).strftime("%Y-%m-%d"))
        for i in range(1, 101)]

df = spark.createDataFrame(data, schema)
df.createOrReplaceTempView("employee_data")

# Show schema and sample
df.printSchema()
df.show(10)

In [0]:
# Aggregations
salary_summary = df.groupBy("department").agg(
    F.avg("salary").alias("avg_salary"),
    F.max("salary").alias("max_salary"),
    F.min("salary").alias("min_salary")
)
salary_summary.show()

# Window function: rank employees by salary per department
windowSpec = Window.partitionBy("department").orderBy(F.desc("salary"))
ranked_df = df.withColumn("rank", F.rank().over(windowSpec))
ranked_df.show()

# Filter top earners per department
top_earners = ranked_df.filter("rank <= 3")
top_earners.show()



In [0]:
# Add calculated column
with_bonus = df.withColumn("bonus", F.col("salary") * 0.1)
with_bonus.show()

# UDF example: Categorize salary band
def salary_band(salary):
    if salary < 50000:
        return "Low"
    elif salary < 90000:
        return "Medium"
    else:
        return "High"

salary_band_udf = udf(salary_band, StringType())
banded_df = df.withColumn("salary_band", salary_band_udf(F.col("salary")))
banded_df.show()

# Join with bonus table
joined_df = banded_df.join(with_bonus.select("id", "bonus"), on="id")
joined_df.show()

# Write to Delta Lake
delta_path = "/tmp/delta/employee_data"
joined_df.write.format("delta").mode("overwrite").save(delta_path)

# Read from Delta and query
delta_df = spark.read.format("delta").load(delta_path)
delta_df.createOrReplaceTempView("delta_employees")

# SQL transformation
high_salary_sql = spark.sql("""
SELECT name, department, salary, bonus
FROM delta_employees
WHERE salary_band = 'High'
ORDER BY salary DESC
""")
high_salary_sql.show()

# Cleanup for re-runs
dbutils.fs.rm(delta_path, recurse=True)
